Neste trabalho iremos fazer a busca dos melhores hiperparametros de uma SVM para Regressão num banco de dados em particular. Para o desenvolvimento utilizaremos a biblioteca sklearn.svm que apresenta a função  SVR() que implementa o regressor SVM e tem vários hiperparametros.
Vamos usar o kernel “rbf”, havendo 3 hiperparametros que consideramos como os mais importantes: C, gamma, e epsilon.

Vamos fazer a busca no range:
    - C entre 2^{-5} e 2^{15}  (uniforme nos expoentes);
    - gamma entre 2^{15} e 2^{-3} (uniforme nos expoentes);
    - episolon entre 0.05 e 10  (uniforme neste intervalo);
    
Utilizamos como dados de treino e testes os arquivos Xtreino5.npy e Xteste5.npy. Para as sáidas dos dados correspondenetres usamos os arquivos ytreino5.npy e yteste5.npy. Todos os arquivos foram disponibilizados na página no trabalho https://www.ic.unicamp.br/~wainer/cursos/1s2020/431/ex4.html.

### Importação das Bibliotecas

Como primeiro passo, importamos as bibliotecas necessárias para a implementação desse trabalho: numpy e sklearn.svm.

In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
import pyspark
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

from pyswarm import pso
import optuna
from scipy.stats import loguniform
import random

### Leitura e Exibição dos arquivos

Após importamos as bibliotecas necessárias, realizamos a leitura dos arquivos .npy disponibilizados para uso neste trabalho. Para isso fizemos uso da função load() da biblioteca numpy. Armazenamos cada um dos arquivos de treino e de teste em variáveis locais.

In [2]:
x_treino = np.load('data/Xtreino5.npy')
x_teste = np.load('data/Xteste5.npy')
y_treino = np.load('data/ytreino5.npy')
y_teste = np.load('data/yteste5.npy')

### Medida de erro

Como primeira tarefa, para cada conjunto de hiperparametros, treinamos o SVM no conjunto de treino (Xtreino e ytreino), e medimos o erro absoluto médio (MAE) no conjunto de teste (Xteste e yteste).

## Randomized Search

In [3]:
n_combinations = 125

# Hyperparameters Search Space
C_range = np.random.uniform(-5, 15, n_combinations).astype(float)
C_range = 2**C_range

# C_range = loguniform(2**-5, 2**15).rvs(size=n_combinations)

gamma_range = np.random.uniform(-15, 3, n_combinations).astype(float)
gamma_range = 2**gamma_range

# gamma_range = loguniform(2**-15, 2** 3).rvs(size=n_combinations)

epsilon_range = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

 
hyperparameters = {'gamma': list(gamma_range), 
                    'C': list(C_range),
                  'epsilon': list(epsilon_range)}
 
# print (hyperparameters)

In [4]:
# Run randomized search
randomCV = RandomizedSearchCV(SVR(kernel='rbf'), param_distributions=hyperparameters, n_iter=20)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']

print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))


The best performing C value is: 188.52
The best performing gamma value is: 0.00009
The best performing epsilon value is:  0.45


In [5]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  3.5487968270739882


### Grid seach

O próximo algoritimo de otimização proposto foi o Grid Search. Seguindo a especificação de uma busca em uma grid de 5x5x5, amostras do range de busca foram tomadas:

In [6]:
# Hyperparameters Search Space
n_combinations = 5

# Taken 5 samples randomly
grid_c = random.sample(list(C_range), k=n_combinations)
grid_gamma = random.sample(list(gamma_range), k=n_combinations)
grid_episolon = random.sample(list(epsilon_range), k=n_combinations)

# The best performing gamma value is: 0.00021
# The best performing C value is: 1425.76
# The best performing epsilon value is:  0.91

# Generate a uniform distribution with 5 elements
# grid_c = np.random.uniform(-5, 15, n_combinations).astype(float)
# grid_c = 2**grid_c

# grid_gamma = np.random.uniform(-15, 3, n_combinations).astype(float)
# grid_gamma = 2**grid_gamma

# grid_episolon = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

# The best performing gamma value is: 0.01466
# The best performing C value is: 593.42
# The best performing epsilon value is:  0.16

hyperparameters_grid = {'gamma': list(grid_gamma), 
                        'C': list(grid_c),
                        'epsilon': list(grid_episolon)}

In [7]:
# Run Grid Search
randomCV = GridSearchCV(SVR(kernel='rbf'), param_grid=hyperparameters_grid, cv = 5)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

The best performing gamma value is: 0.00058
The best performing C value is: 49.70
The best performing epsilon value is:  0.77


In [8]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  3.6237947770473338


### Otimização bayesiana

Outro algoritimo proposto foi a otimização bayesiana. Para a sua implementação utilziamos a biblioteca hyperopt, que dispnibiliza O regressor (TPE) para modelar a distribuição de probabilidades que é muito mais rápido que a implementação padrão utilizando “processos gaussianos”.

Esta implementação foi feita em passos:

### Definir a Função de mínimo

Como queremos pesquisar por Support Vector Machines (SVM), definimos um parâmetro params ['type'] como o nome do modelo, e uma função para executar o treinamento e retornar a precisão da validação cruzada. 
Como estamos tentando maximizar a precisão da validação cruzada, devemos negar esse valor para o hyperopt, pois o hyperopt sabe apenas como minimizar uma função.

In [9]:
def get_acc_status(clf,X_,y):
    acc = cross_val_score(clf, X_, y, cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [10]:
def objective(params):
    classifier_type = params['type']
    del params['type']
    if classifier_type == 'svm':
        clf = SVR(**params)
    else:
        return 0
    accuracy = cross_val_score(clf, x_treino, y_treino).mean()
    
    return {'loss': -accuracy, 'status': STATUS_OK}

### Definir espaço de pesquisa sobre os hiperparâmetros

In [11]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.uniform('C', (2**-5), (2**15)),
        'gamma': hp.uniform('gamma', (2**-15), (2**3)),
        'epsilon': hp.uniform('epsilon', 0.05, 1.0),
        'kernel': hp.choice('kernel', ['rbf'])
    },
])

### Selecionar um algoritimo de busca

As duas opções principais de algoritmos de busca são:

    - hyperopt.tpe.suggest: Estimadores da Árvore de Parzen, uma abordagem bayesiana que seleciona iterativa e adaptativamente novas configurações de hiperparâmetro para explorar com base em resultados anteriores;
    - hyperopt.rand.suggest: Pesquisa aleatória, uma abordagem não adaptativa que mostra o espaço de pesquisa.
    
Conforme pedido, utilziamos o algoritimo TPE.

### Executar o algoritmo de ajuste com hyperopt fmin ()

Definimos max_evals como o número máximo de pontos no espaço do hiperparâmetro para testar, ou seja, o número máximo de modelos para ajustar e avaliar.

O SparkTrials usa 2 argumentos:
    - parallelism: Número de modelos para ajustar e avaliar simultaneamente.
    - timeout: tempo máximo (em segundos) que fmin pode demorar. Este argumento é opcional.

O rastreamento automatizado do MLflow está ativado por padrão. Ligue para mlflow.start_run () antes de chamar fmin (), como mostra o exemplo abaixo.

In [12]:
hypopt_trials = Trials()
 
best_params = fmin(objective, search_space, algo=tpe.suggest, 
max_evals=125, trials= hypopt_trials)
 
best_gamma  = best_params['gamma']
best_C      = best_params['C']
best_epsilon= best_params['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

100%|██████████| 125/125 [00:20<00:00,  5.98trial/s, best loss: -0.512377857326078]  
The best performing gamma value is: 0.00383
The best performing C value is: 23889.46
The best performing epsilon value is:  0.35


In [13]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  4.424817673220065


## PSO

In [ ]:
# PARAMETERS
C_MIN = 2**(-5)
C_MAX = 2**15

GAMMA_MIN = 2**(-15)
GAMMA_MAX = 2**3

EPSILON_MIN = 0.05
EPSILON_MAX = 1.0

lb = np.array([C_MIN, GAMMA_MIN, EPSILON_MIN])
ub = np.array([C_MAX, GAMMA_MAX, EPSILON_MAX])

# FUNCTION
def svr_fun(X):
    c = X[0]
    g = X[1]
    eps = X[2]
    
    svr  = SVR(kernel='rbf', C=c, gamma=g, epsilon=eps)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae

print("PSO...")
x_opt, y_opt = pso(svr_fun, lb, ub, swarmsize=11, maxiter=11)

print(" C optimal: "+ str(x_opt[0])+
     "\n Gamma Optimal: "+ str(x_opt[1])+
     "\n Epsilon Optimal: "+ str(x_opt[2]))
print("MAE: ", str(y_opt))

PSO...


## Simulated Annealing

Para a implementação dos Métodos de Simmulate Annealing e CMA-ES foi utilizado a biblioteca optuna.

Para o método de Simulated Annealing a biblioteca fornece uma implementação via classe através dos chamados "samplers" para determinar os valores dos parâmetros a serem avaliados durante o teste. Para o Simmulated Annealing o sampler é implementado via classe explicitamente (vale ressaltar que estamos utilizando a implementação proposta pela própria documentação).

In [ ]:
class SimulatedAnnealingSampler(optuna.samplers.BaseSampler):
    def __init__(self, temperature=100):
        self._rng = np.random.RandomState()
        self._temperature = temperature  # Current temperature.
        self._current_trial = None  # Current state.

    def sample_relative(self, study, trial, search_space):
        if search_space == {}:
            return {}

        #
        # An implementation of SA algorithm.
        #

        # Calculate transition probability.
        prev_trial = study.trials[-2]
        if self._current_trial is None or prev_trial.value <= self._current_trial.value:
            probability = 1.0
        else:
            probability = np.exp((self._current_trial.value - prev_trial.value) / self._temperature)
        self._temperature *= 0.9  # Decrease temperature.

        # Transit the current state if the previous result is accepted.
        if self._rng.uniform(0, 1) < probability:
            self._current_trial = prev_trial

        # Sample parameters from the neighborhood of the current point.
        #
        # The sampled parameters will be used during the next execution of
        # the objective function passed to the study.
        params = {}
        for param_name, param_distribution in search_space.items():
            if not isinstance(param_distribution, optuna.distributions.UniformDistribution):
                raise NotImplementedError('Only suggest_uniform() is supported')

            current_value = self._current_trial.params[param_name]
            width = (param_distribution.high - param_distribution.low) * 0.1
            neighbor_low = max(current_value - width, param_distribution.low)
            neighbor_high = min(current_value + width, param_distribution.high)
            params[param_name] = self._rng.uniform(neighbor_low, neighbor_high)

        return params

    #
    # The rest is boilerplate code and unrelated to SA algorithm.
    #
    def infer_relative_search_space(self, study, trial):
        return optuna.samplers.intersection_search_space(study)

    def sample_independent(self, study, trial, param_name, param_distribution):
        independent_sampler = optuna.samplers.RandomSampler()
        return independent_sampler.sample_independent(study, trial, param_name, param_distribution)





Por fim o é necessário criar uma função objetivo a qual deseja-se otimizar, em nosso caso procuramos minimizar o Erro Médio Absoluto (MAE) com uma validação no conjunto de testes da aplicação dos hyperparâmetros encontrados para o problema do SVM Regressor.

In [ ]:
def objective(trial):
    c = trial.suggest_uniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_uniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae


sampler = SimulatedAnnealingSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=125)

Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [ ]:
study.best_params

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [ ]:
study.best_value

## CMA-ES

Semelhantemente ao algoritmo de Simullate Annealing, a implementação do algoritmo de otimização CMA-ES também foi feita via biblioteca optunza. A implementação porém é encapsulada em um sampler default fornecido pela própria biblioteca, assim não sendo necessário a implementação de uma classe sampler específica, somente a função objetivo.

In [ ]:
def objective(trial):
    c = trial.suggest_uniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_uniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae


sampler = optuna.samplers.CmaEsSampler()
study = optuna.create_study(sampler=sampler)
study.optimize(objective, n_trials=125)


Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [ ]:
study.best_params

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [ ]:
study.best_value